# TPI Teoría de Control: Análisis de Simulación del Rate Limiter

**Alumno:** Matías Ezequiel Nuñez

Este notebook importa el simulador de `Token Bucket` y ejecuta dos escenarios clave para validar nuestro modelo de control (PI No Lineal).

## 1. Configuración e Importaciones

Primero, importamos las bibliotecas necesarias y el motor de simulación que se encuentra en `sim/token_bucket.py`.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sim.token_bucket import TokenBucketSimulator

# Configuración para que Matplotlib se vea bien en el notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

# --- Parámetros Globales de Simulación ---

# R(s) - Setpoint (Referencia): Tasa de generación de tokens
R_RATE = 100.0  # tokens/segundo

# B - Capacidad del Bucket: Define la capacidad de absorber ráfagas
B_CAPACITY = 200.0 # tokens

# dt - Resolución de la simulación
DT = 0.01 # segundos (10ms)

# Instanciamos el simulador
simulator = TokenBucketSimulator(R_rate=R_RATE, B_capacity=B_CAPACITY, dt=DT)

print(f"Simulador inicializado:")
print(f"  Referencia (R): {R_RATE} req/s")
print(f"  Capacidad Ráfaga (B): {B_CAPACITY} tokens")

Simulador inicializado:
  Referencia (R): 100.0 req/s
  Capacidad Ráfaga (B): 200.0 tokens


## 2. Escenario 1: Análisis de Respuesta Transitoria (Ráfagas)

**Objetivo:** Analizar la **Respuesta Transitoria** del sistema.

Vamos a simular un tráfico normal (50 req/s) que de repente sufre un pico (ráfaga) a 300 req/s. Esperamos ver cómo el sistema usa los tokens acumulados (`B`) para absorber la ráfaga, permitiendo momentáneamente una salida `Y(t) > R(t)`.

In [ ]:
# --- Definición del Patrón de Tráfico 1 --- 
duration_sec = 20
n_steps = int(duration_sec / DT)
time_vector = np.arange(0, duration_sec, DT)

traffic_1 = np.full(n_steps, 50.0) # Carga base

# Ráfaga 1 (5s a 7s)
traffic_1[(time_vector >= 5) & (time_vector < 7)] = 300.0

# Ráfaga 2 (12s a 15s)
traffic_1[(time_vector >= 12) & (time_vector < 15)] = 150.0

# --- Ejecución de la Simulación 1 ---
results_1 = simulator.simulate(traffic_1)

print("Simulación Escenario 1 completada.")

In [ ]:
# --- Graficación Escenario 1 ---

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
fig.suptitle('Escenario 1: Respuesta Transitoria a Ráfagas', fontsize=16)

# Gráfico Superior: Tasas de Requests
ax1.plot(results_1["time"], results_1["incoming_rate"], 'orange', linestyle='--', label='Tasa de Llegada (Perturbación D(t))')
ax1.plot(results_1["time"], results_1["allowed_rate"], 'b-', label='Tasa Permitida (Salida Y(t))')
ax1.plot(results_1["time"], results_1["rejected_rate"], 'r:', alpha=0.7, label='Tasa Rechazada')
ax1.axhline(y=R_RATE, color='k', linestyle=':', label=f'Setpoint (R = {R_RATE} req/s)')
ax1.set_ylabel('Tasa de Requests (req/s)')
ax1.legend()
ax1.grid(True)

# Gráfico Inferior: Nivel de Tokens (La Integral)
ax2.plot(results_1["time"], results_1["token_levels"], 'g-', label='Nivel de Tokens (Integral de Error)')
ax2.axhline(y=B_CAPACITY, color='k', linestyle=':', label=f'Capacidad Máxima (B = {B_CAPACITY})')
ax2.set_xlabel('Tiempo (s)')
ax2.set_ylabel('Tokens Acumulados')
ax2.legend()
ax2.grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

#### Análisis de Resultados (Escenario 1)

1.  **Periodo 0-5s:** La Tasa de Llegada (50 req/s) es menor que la Referencia (100 req/s). El error `E = R - Y` es positivo. La acción integral acumula este error positivo y el balde se llena hasta su Capacidad Máxima (B=200). 
2.  **Periodo 5-7s (Ráfaga):** La Tasa de Llegada (300 req/s) supera la referencia. 
    * **Respuesta Transitoria:** El sistema usa los 200 tokens acumulados para absorber la ráfaga. Notar cómo la Tasa Permitida `Y(t)` (azul) **supera** el Setpoint `R(t)` (negro) durante el primer segundo del pico. Esto es la *capacidad de ráfaga*.
    * **Saturación:** El balde (la integral) se vacía (`Tokens = 0`). En este punto, la acción Proporcional (ON/OFF) se activa en modo "OFF" (saturado). El sistema ahora solo permite la Tasa de Referencia (100 req/s) y rechaza 200 req/s.
3.  **Periodo 7-12s (Recuperación):** La carga vuelve a 50 req/s. El error `E = R - Y` vuelve a ser positivo. La acción integral recarga el balde, preparándolo para la próxima ráfaga.

## 3. Escenario 2: Análisis de Estabilidad y Error en Estado Estacionario (Ataque DoS)

**Objetivo:** Analizar la **Estabilidad** y el **Error en Estado Estacionario ($e_{ss}$)**.

Simulamos una perturbación de escalón masiva y sostenida (Ataque DoS) de 500 req/s. Queremos validar dos cosas:
1.  **Estabilidad:** La salida `Y(t)` no debe divergir, debe estabilizarse (saturar).
2.  **Error Estacionario:** Debemos calcular `e_ss = R - Y`.

In [ ]:
# --- Definición del Patrón de Tráfico 2 --- 
duration_sec = 20
n_steps = int(duration_sec / DT)
time_vector = np.arange(0, duration_sec, DT)

traffic_2 = np.full(n_steps, 50.0) # Carga base

# Ataque DoS Sostenido (5s a 15s)
traffic_2[(time_vector >= 5) & (time_vector < 15)] = 500.0

# --- Ejecución de la Simulación 2 ---
results_2 = simulator.simulate(traffic_2)

print("Simulación Escenario 2 completada.")

In [ ]:
# --- Graficación Escenario 2 ---

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
fig.suptitle('Escenario 2: Estabilidad bajo Ataque DoS', fontsize=16)

# Gráfico Superior: Tasas de Requests
ax1.plot(results_2["time"], results_2["incoming_rate"], 'orange', linestyle='--', label='Tasa de Llegada (Perturbación D(t))')
ax1.plot(results_2["time"], results_2["allowed_rate"], 'b-', label='Tasa Permitida (Salida Y(t))')
ax1.plot(results_2["time"], results_2["rejected_rate"], 'r:', alpha=0.7, label='Tasa Rechazada (Perturbación Rechazada)')
ax1.axhline(y=R_RATE, color='k', linestyle=':', label=f'Setpoint (R = {R_RATE} req/s)')
ax1.set_ylabel('Tasa de Requests (req/s)')
ax1.legend()
ax1.grid(True)

# Gráfico Inferior: Nivel de Tokens (La Integral)
ax2.plot(results_2["time"], results_2["token_levels"], 'g-', label='Nivel de Tokens (Integral de Error)')
ax2.axhline(y=B_CAPACITY, color='k', linestyle=':', label=f'Capacidad Máxima (B = {B_CAPACITY})')
ax2.set_xlabel('Tiempo (s)')
ax2.set_ylabel('Tokens Acumulados')
ax2.legend()
ax2.grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
#### Análisis de Resultados (Escenario 2)

1.  **Estabilidad:** En `t=5s`, la perturbación (500 req/s) golpea el sistema. Tras una breve respuesta transitoria (donde se consumen los 200 tokens de `B`), el balde se vacía (`Tokens = 0`) y permanece vacío. La salida del sistema `Y(t)` (azul) **no diverge**, sino que **se estabiliza (satura) exactamente en el valor del Setpoint `R(t)`** (100 req/s). El sistema es, por definición, **estable**.

2.  **Error en Estado Estacionario ($e_{ss}$):** Este es el punto central del TPI.
    * La señal de referencia es $R(t) = 100$ req/s (Tasa de generación de tokens).
    * La variable controlada (salida) en estado estacionario (durante el ataque, de `t=5.5s` a `t=15s`) es $Y(t) = 100$ req/s (Tasa permitida).
    * El error del sistema de control es: $e_{ss} = R(t) - Y(t) = 100 - 100 = 0$.

    **Conclusión del Análisis:** El sistema de control tiene un **Error en Estado Estacionario NULO**. Esto valida nuestra teoría: el componente Integral (el balde) anula el error en estado estacionario, tal como lo predice la teoría de control para un sistema de **Tipo 1** ante una entrada de escalón.

3.  **¿Qué son las 400 req/s rechazadas?** Es fundamental entender que las 400 req/s (línea roja punteada) **NO SON EL ERROR** del sistema de control. Son la **PERTURBACIÓN ($D(t)$) que ha sido exitosamente rechazada** por la acción de control. El controlador está cumpliendo su objetivo a la perfección: proteger la planta (el microservicio) asegurando que `Y(t)` sea igual a `R(t)`.